# Notebook for preparing necessary data for COM6101 group project

## Introduction

We want to start an italian restaurant in Hong Kong. We want to find the best location for our restaurant. as well as the menu we should offer. (to be added more details)

## Data Preparation

Here are the data we need to prepare for our project:
* 2021 Hong Kong Census data (source: https://data.gov.hk/en-data/dataset/hk-censtatd-census_geo-2021-population-census-by-ssg )
* Hong Kong Restaurant License (source: https://data.gov.hk/en-data/dataset/hk-fehd-fehdlmis-restaurant-licences )
* Hong Kong Pedestrian Network (source: https://opendata.esrichina.hk/datasets/48e295256fd84032a87b27000cea35cd/about)
* Hong Kong pedestrian network (souce: https://portal.csdi.gov.hk/geoportal/#metadataInfoPanel )
* <span style="background-color: #FF0000">Hong Kong Geocommunity (source: https://data.gov.hk/en-data/dataset/hk-landsd-openmap-development-hkms-digital-geocom/resource/0931b84b-f1c8-409f-9cd4-176a26645db0 ) (May not useful, as the data cannot match the time)</span>

## Data Processing

In this notebook, we will process the data to get the following information:

### Restaurant License Data

In [4]:
# transfer xml to csv for restaurant licence data
import os
import xml.etree.ElementTree as Xet
import pandas as pd

folder_path = 'data/restaurant_license/xml'
output_folder = 'data/restaurant_license/csv'

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

cols = ["TYPE", "DIST", "LICNO", "SS", "ADR", "INFO", "EXPDATE"]

# Iterate through all XML files in the folder_path
for filename in os.listdir(folder_path):
    if filename.endswith(".XML"):
        xml_file = os.path.join(folder_path, filename)
        csv_file = os.path.join(output_folder, os.path.splitext(filename)[0][1:] + ".csv")
        rows = []
        xmlparse = Xet.parse(xml_file)
        # The data are stored in the <LPS> tag
        root = xmlparse.getroot().find("LPS")

        for i in root:
            
            TYPE = i.find("TYPE").text
            DIST = i.find("DIST").text
            LICNO = i.find("LICNO").text
            SS = i.find("SS").text
            ADR = i.find("ADR").text
            INFO = i.find("INFO").text
            EXPDATE = i.find("EXPDATE").text
            rows.append({"TYPE": TYPE, "DIST": DIST, "LICNO": LICNO, 
                         "SS": SS, "ADR": ADR, "INFO": INFO, "EXPDATE": EXPDATE})

        df = pd.DataFrame(rows, columns=cols)
        df.to_csv(csv_file, index=False)
        print(f"Converted {filename} to {os.path.splitext(filename)[0][1:] + '.csv'}")

print("XML to CSV conversion completed.")

Converted 20230101-1031-LP_Restaurants_TC.XML to 0230101-1031-LP_Restaurants_TC.csv
Converted 20240101-1108-LP_Restaurants_TC.XML to 0240101-1108-LP_Restaurants_TC.csv
XML to CSV conversion completed.


In [ ]:
# geocoding for restaurant licence data
import os
import pandas as pd
import geocoder

API_KEY = "AAPK3ac33a03387e472db4ac246ec302438c69Oee1EeJzIYy8Qlsbogx5ZziXUuE247Xeb2ASN4douVyGLHZ9MwCGHEtPdJPJz8"

folder_path = 'restaurant-info-hk/csv'
output_folder = 'restaurant-info-hk/geo'

# create output folder if not exist
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

# Iterate through all CSV files in the folder_path
# columns: TYPE, DIST, LICNO, SS, ADR, INFO, EXPDATE
# column ADR is the address

# use old csv file to reference if address already exist
old_file = 'restaurant-info-hk/geo/old_file.csv'

for filename in os.listdir(folder_path):
    # only retrive csv file and file not end with .finished.csv
    if filename.endswith(".csv") and not filename.endswith(".finished.csv"):
        print(f'start converting {filename} to geo location')
        csv_file = os.path.join(folder_path, filename)
        geo_file = os.path.join(output_folder, filename + ".geo.csv")
        # find rows if ADR contains ['KWAI CHUNG"] or ['TSUEN WAN']
        # transform ADR to upper case first
        df = pd.read_csv(csv_file, encoding='utf-8')
        df['ADR'] = df['ADR'].str.upper()
        df = df.loc[lambda df: df['ADR'].str.contains('KWAI CHUNG') | df['ADR'].str.contains('TSUEN WAN')]
        # add new columns for geo location
        lat = []
        lng = []
        df.reset_index(inplace=True, drop=True)
        # check if df['ADR'] == old_df['ADR']
        old_df = pd.read_csv(old_file, encoding='utf-8')
        for i in range(len(df)):
            if df['ADR'][i] in old_df['ADR'].values:
                # if address already exist in old_df, then use the geo location in old_df
                lat.append(old_df.loc[old_df['ADR'] == df['ADR'][i], 'lat'].iloc[0])
                lng.append(old_df.loc[old_df['ADR'] == df['ADR'][i], 'lng'].iloc[0])
                print(f"Converted {df['ADR'][i]} to {old_df.loc[old_df['ADR'] == df['ADR'][i], 'lat'].iloc[0]}, {old_df.loc[old_df['ADR'] == df['ADR'][i], 'lng'].iloc[0]}, by referencing old csv file")
                continue
            g = geocoder.arcgis(df['ADR'][i], key = API_KEY)
            lat.append(g.lat)
            lng.append(g.lng)
            print(f"Converted {df['ADR'][i]} to {g.lat}, {g.lng}")
        df['lat'] = lat
        df['lng'] = lng
        df.to_csv(geo_file, index=False)
        # rename the original csv file to finished
        os.rename(csv_file, os.path.join(folder_path, filename + ".finished.csv"))
        print(f"Converted {filename} to {filename}")
print("All csv files converted to geo location")